In [1]:
# 必要なライブラリのimport
from datasets import load_dataset
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

In [2]:
# 今回利用するデータセットのダウンロード
dataset = load_dataset("shunk031/wrime", name="ver1", trust_remote_code=True)
dataset

DatasetDict({
    train: Dataset({
        features: ['sentence', 'user_id', 'datetime', 'writer', 'reader1', 'reader2', 'reader3', 'avg_readers'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['sentence', 'user_id', 'datetime', 'writer', 'reader1', 'reader2', 'reader3', 'avg_readers'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['sentence', 'user_id', 'datetime', 'writer', 'reader1', 'reader2', 'reader3', 'avg_readers'],
        num_rows: 2000
    })
})

In [3]:
# データセットをPandasで扱えるフォーマットに変換して、trainデータの先頭5件を表示
dataset.set_format(type="pandas")
# CPUでの実行時間を考慮し、訓練データ数を調整 (例: 5000件)
# もしメモリや時間に余裕があれば、より多くのデータを使用することも可能です。
num_train_samples = 5000
train_df = dataset["train"][:num_train_samples]

train_df.head()

,sentence,user_id,datetime,writer,reader1,reader2,reader3,avg_readers
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1,2012/07/31 23:48,"{'joy': 0, 'sadness': 1, 'anticipation': 2, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's..."
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,1,2012/08/02 23:09,"{'joy': 3, 'sadness': 0, 'anticipation': 3, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's..."
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,1,2012/08/05 00:50,"{'joy': 1, 'sadness': 1, 'anticipation': 1, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's..."
3,眠い、眠れない。,1,2012/08/08 01:36,"{'joy': 0, 'sadness': 2, 'anticipation': 1, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's..."
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,1,2012/08/09 22:24,"{'joy': 2, 'sadness': 1, 'anticipation': 3, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's..."


In [4]:
# trainデータの最初の'avg_readers'を確認
train_df.loc[0, "avg_readers"]

{'joy': 0,
 'sadness': 2,
 'anticipation': 0,
 'surprise': 0,
 'anger': 0,
 'fear': 0,
 'disgust': 0,
 'trust': 0}

In [5]:
# 感情のkeyをリストにする
em_keys = list(train_df.loc[0, "avg_readers"].keys())

In [6]:
# それぞれのDataFrameに'emotion'列を追加し、数値が一番大きいavg_readersのkeyを格納する
train_df["emotion"] = [em_keys.index(max(em, key=em.get)) for em in train_df["avg_readers"]]

In [7]:
# sentence列とemotion列のみ残す
train_df = train_df.drop(["user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"], axis=1)
train_df.head()

,sentence,emotion
0,ぼけっとしてたらこんな時間｡チャリあるから食べにでたいのに…,1
1,今日の月も白くて明るい。昨日より雲が少なくてキレイな? と立ち止まる帰り道｡チャリなし生活も...,3
2,早寝するつもりが飲み物がなくなりコンビニへ｡ん､今日、風が涼しいな。,3
3,眠い、眠れない。,1
4,ただいま? って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!,0


In [8]:
# emotionの内訳を確認
train_df.groupby("emotion").count()

,sentence
emotion,
0,1421
1,1100
2,1064
3,887
4,124
5,139
6,231
7,34


In [9]:
# シンボリックリンクの警告を表示させない
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

# sentenceの文章をトークナイザーで前処理する
# 軽量な多言語対応モデルのトークナイザーを指定
tokenizer_name = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)

tokenized_data = tokenizer(
    train_df["sentence"].to_list(),
    return_tensors="np",  # TensorFlow形式のテンソルで返す
    padding=True,         # 短いシーケンスをパディング
    truncation=True,      # 長いシーケンスを切り捨て
    max_length=512        # モデルが扱える最大長に設定 (多くのモデルで一般的)
)
tokenized_data = dict(tokenized_data)  # 辞書形式に変換しておく
tokenized_data

{'input_ids': array([[  101,  1960, 27849, ...,     0,     0,     0],
        [  101,  2187,  4348, ...,     0,     0,     0],
        [  101,  4352,  3425, ...,     0,     0,     0],
        ...,
        [  101,  5915,  4704, ...,     0,     0,     0],
        [  101,  2531,  2046, ...,     0,     0,     0],
        [  101,   124,  4388, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [10]:
# emotion列をndarrayにする
emotions = np.array(train_df["emotion"])

In [ ]:
# モデルを作成して学習させる
model_name = "distilbert-base-multilingual-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=8)

# オプティマイザや学習率を設定
# 軽量モデルやCPU実行の場合、学習率やバッチサイズの調整が効果的なことがあります。
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5) # 一般的な学習率
# model.compileのlossは、TFAutoModelForSequenceClassification.from_pretrainedでfrom_pt=TrueとしてPyTorchの重みをロードした場合や、
# カスタム損失関数を使わない場合は、通常モデル内部で処理されるか、フレームワーク標準の損失関数が使われます。
# Hugging FaceのTFモデルでは、loss引数をcompile時に指定せず、fit時に直接ラベルを渡すことで内部損失が使われることが多いです。
# ここでは明示的に`model.hf_compute_loss`を使っていますが、もしこれが原因で別のエラーが出る場合は、
# `loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)` のような標準的な損失関数を試すことも検討できます。
# ただし、Hugging Faceのドキュメントでは `model.hf_compute_loss` または `loss=model.hf_compute_loss()` の使用が推奨される場合があります。
model.compile(optimizer=optimizer, loss=model.hf_compute_loss, metrics=['accuracy']) # metricsを追加して精度を監視

# バッチサイズとエポック数を設定
# CPU実行ではバッチサイズを小さめ (例: 8, 16) に、エポック数も少なめ (例: 1-3) にすると時間短縮になります。
batch_size = 16
epochs = 1 # まずは1エポックで試すなど、調整してください

print("ファインチューニングを開始します。CPUでは時間がかかる場合があります...")
# tokenized_dataは辞書形式である必要があります。{'input_ids': ..., 'attention_mask': ...}
# emotionsはラベルのnumpy配列です。
model.fit(dict(tokenized_data), emotions, batch_size=batch_size, epochs=epochs)
print("ファインチューニングが完了しました。")

In [12]:
# testデータについても同様に前処理をしていく
test_df = dataset["test"][:]
test_df.head()

,sentence,user_id,datetime,writer,reader1,reader2,reader3,avg_readers
0,汗めっちゃかいた(),49,2016/06/18 15:25,"{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's..."
1,1人だけ春みたいなかっこしててはずい,49,2016/06/18 15:40,"{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's..."
2,……はあ；；；；,49,2016/06/18 21:36,"{'joy': 2, 'sadness': 0, 'anticipation': 2, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's..."
3,あぁーテスト勉強(),49,2016/06/19 21:17,"{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 1, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 0, 'sadness': 2, 'anticipation': 0, 's..."
4,ハシが狂っていく様が儚げで見てて辛かったけど、なんか好き…,49,2016/06/19 22:14,"{'joy': 1, 'sadness': 0, 'anticipation': 1, 's...","{'joy': 1, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 2, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 0, 'anticipation': 0, 's...","{'joy': 2, 'sadness': 1, 'anticipation': 0, 's..."


In [13]:
test_df["emotion"] = [em_keys.index(max(em, key=em.get)) for em in test_df["avg_readers"]]
test_df = test_df.drop(["user_id", "datetime", "writer", "reader1", "reader2", "reader3", "avg_readers"], axis=1)
test_df.head()

,sentence,emotion
0,汗めっちゃかいた(),0
1,1人だけ春みたいなかっこしててはずい,1
2,……はあ；；；；,0
3,あぁーテスト勉強(),1
4,ハシが狂っていく様が儚げで見てて辛かったけど、なんか好き…,0


In [14]:
test_tokenized_data = tokenizer(
    test_df["sentence"].to_list(),
    return_tensors="np",
    padding=True,
    truncation=True,
    max_length=512
)
test_tokenized_data = dict(test_tokenized_data)
test_tokenized_data

{'input_ids': array([[  101,  4881,  1965, ...,     0,     0,     0],
        [  101,   122,  2179, ...,     0,     0,     0],
        [  101,   100,   100, ...,     0,     0,     0],
        ...,
        [  101,  2195,  2149, ...,     0,     0,     0],
        [  101,  4422,  1925, ...,     0,     0,     0],
        [  101,  2187, 10906, ...,     0,     0,     0]]),
 'attention_mask': array([[1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        ...,
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0],
        [1, 1, 1, ..., 0, 0, 0]])}

In [15]:
test_emotions = np.array(test_df["emotion"])

In [16]:
# testデータで予測を実施
print("テストデータで予測を開始します...")
predictions = model.predict(test_tokenized_data)
print("予測が完了しました。")

テストデータで予測を開始します...
63/63 [==============================] - 140s 2s/step
予測が完了しました。


In [17]:
# 実行結果をそのまま表示してみる
predictions

TFSequenceClassifierOutput(loss=None, logits=array([[ 1.019193  ,  1.6175576 , -0.0738837 , ..., -0.70726997,
        -0.25972995, -2.1955812 ],
       [ 0.30437595,  1.6098711 ,  0.1044722 , ..., -0.56013715,
         0.14716396, -2.1297877 ],
       [ 0.4930316 ,  1.5993042 ,  0.02099986, ..., -0.64035827,
         0.08014511, -2.1811657 ],
       ...,
       [ 2.6014364 ,  0.5489538 ,  1.3552492 , ..., -1.3935866 ,
        -1.0788763 , -2.3455694 ],
       [ 0.37253407,  1.6692158 , -0.24800672, ..., -0.39635023,
         0.12506332, -2.0484025 ],
       [ 2.6210232 ,  0.66216105,  1.3252009 , ..., -1.519871  ,
        -1.120725  , -2.5914352 ]], dtype=float32), hidden_states=None, attentions=None)

In [18]:
# 一番大きい確率のものを予測結果とする形で調整する
# model.predictの出力がTFSequenceClassifierOutputオブジェクトの場合、.logitsでアクセス
pred_emotions = np.argmax(predictions.logits, axis=1)
pred_emotions

array([1, 1, 1, ..., 0, 1, 0])

In [19]:
# 実際の値を確認
test_emotions

array([0, 1, 0, ..., 2, 6, 2])

In [20]:
from sklearn import metrics

# 混同行列
print("混同行列:")
print(metrics.confusion_matrix(test_emotions, pred_emotions))

混同行列:
[[328 244   9   6   0   0   7   0]
 [ 47 325   2   4   0   0  15   0]
 [150 302  45   9   0   0   7   0]
 [ 34 125   3  21   0   0   3   0]
 [  3  16   1   0   0   0   2   0]
 [ 10 161   3   4   0   0   8   0]
 [  8  78   0   3   0   0  10   0]
 [  0   7   0   0   0   0   0   0]]


In [21]:
# 正解率
accuracy = metrics.accuracy_score(test_emotions, pred_emotions)
print(f"正解率: {accuracy:.4f}")

正解率: 0.3645
